In [1]:
# Import Necessary Libraries
import open3d as o3d
import numpy as np
from pc_skeletor.laplacian import SLBC
from scipy.spatial import cKDTree
import time
import laspy
import os
import copy
from math import ceil
import matplotlib.pyplot as plt
from multiprocessing import Pool, cpu_count
from math import ceil
from sklearn.cluster import KMeans
import vtk

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


# Flower Cluster Extraction

In [10]:
# Nearest neighbor search removes the stem point cloud
def remove_nearest_points(source_point_cloud, target_point_cloud, distance_threshold=0.02):
    # Create a KDTree
    kdtree_target = o3d.geometry.KDTreeFlann(target_point_cloud)
    
    correspondences = []
    corresponding_colors = []
    # # Iterate over each point in the source point cloud
    for idx in range(len(source_point_cloud.points)):
        point_source = np.asarray(source_point_cloud.points[idx])
        # Perform radius search in the KDTree to find points within the distance threshold
        [_, idx_b, _] = kdtree_target.search_radius_vector_3d(point_source, distance_threshold)
        correspondences.append(idx_b)
        corresponding_colors.append(np.asarray(target_point_cloud.colors)[np.asarray(idx_b).flatten()])
        
    correspondences = np.hstack(correspondences)
    correspondences = np.array(correspondences).flatten()
    
    # Find the complement of the indices in the target point cloud
    all_indices = set(np.arange(len(target_point_cloud.points)))
    subset_indices = set(correspondences)
    complement_indices = all_indices - subset_indices
    complement_list = list(complement_indices)

    # Select the complement indices to create the filtered target point cloud
    filtered_target_point_cloud = target_point_cloud.select_by_index(complement_list)
    
    return filtered_target_point_cloud

In [11]:
# Color threshold removes stem point clouds
def remove_by_color(point_cloud, color_threshold=0.8):
    colors = np.asarray(point_cloud.colors)
    # Convert RGB colors to grayscale
    gray_colors = []
    for color in colors:
        gray = color[0]*0.299 + color[1]*0.587 + color[2]*0.114
        gray_colors.append([gray, gray, gray])
    
    # Create a new point cloud with grayscale colors
    graypcd = o3d.geometry.PointCloud()
    graypcd.points = o3d.utility.Vector3dVector(point_cloud.points)
    graypcd.colors = o3d.utility.Vector3dVector(gray_colors)
    
    color_array = np.asarray(graypcd.colors)
    selected_indices = np.where(color_array > color_threshold)[0]
    selected_points = graypcd.select_by_index(selected_indices)
    return selected_points

# Function to perform DBSCAN clustering on a point cloud
def dbscan_cluster(point_cloud, output_folder):
    # Perform DBSCAN clustering on the point cloud
    with o3d.utility.VerbosityContextManager(o3d.utility.VerbosityLevel.Debug) as cm:
        labels = np.array(point_cloud.cluster_dbscan(eps=0.02, min_points=20, print_progress=False))
    max_label = labels.max()
    
    j = 1
    for i in range(max_label + 1):
        indices = np.where(labels == i)[0]
        # Save the cluster to a file if it contains at least 50 points
        if len(indices) >= 50:
            cluster_point_cloud = point_cloud.select_by_index(indices)
            file_name = f"Dbscan_cluster{j}.pcd"
            file_path = os.path.join(output_folder, file_name)
            o3d.io.write_point_cloud(file_path, cluster_point_cloud)
            j += 1
            
    colors = plt.get_cmap("tab20")(labels / (max_label if max_label > 0 else 1))
    colors[labels < 0] = 0
    point_cloud.colors = o3d.utility.Vector3dVector(colors[:, :3])
    return point_cloud

In [12]:
# classify data using K-means clustering
def kmeans_classify(trait_data, n_clusters=2):
    kmeans = KMeans(n_clusters=n_clusters, n_init=10, random_state=42)
    kmeans.fit(trait_data)

    kmeans_labels = kmeans.labels_
    # cluster_centers = kmeans.cluster_centers_
    return kmeans_labels